# Week 2. Problem set

## Evgeny Bobkunov SD-03 e.bobkunov@innopolis.university

### 1. Implement the following functions over lists of symbols in Racket using **explicit recursion** (i.e. without using higher-order functions like apply, map, andmap, ormap, filter, and foldl). Each function must be implemented independently. Use tail recursion whenever it helps produce a more efficient implementation:

#### (a) Find the most frequent symbol in a list (return the first one when ambiguous):


```racket
(most-frequent '(h e l l o w o r l d)) ; ==> 'l
```

In [56]:
(define (most-frequent lst)
  (define (count-occurrences lst)
    (define (count-helper lst acc)
      (if (null? lst)
          acc
          (count-helper (cdr lst)
                        (hash-update acc (car lst) add1 0))))
    (count-helper lst (hash)))
  
  (define (find-most-frequent counts)
    (define (find-helper lst max-sym max-count)
      (if (null? lst)
          max-sym
          (let* ([current-sym (car lst)]
                 [current-count (hash-ref counts current-sym)])
            (if (> current-count max-count)
                (find-helper (cdr lst) current-sym current-count)
                (find-helper (cdr lst) max-sym max-count)))))
    (find-helper (hash-keys counts) (car lst) (hash-ref counts (car lst))))
  
  (if (null? lst)
      #f
      (find-most-frequent (count-occurrences lst))))

(display (most-frequent '(h e l l o w o r l d)))

l

#### (b) Annotate each symbol in a list with its occurrence in the list:

```
(annotate-occurrence '(h e l l o w o r l d))
; ==> '((h 1) (e 1) (l 1) (l 2) (o 1) (w 1) (o 2) (r 1) (l 3) (d 1))
```

In [61]:
(define (annotate-occurrence lst)
  (define (annotate-helper lst counts result)
    (if (null? lst)
        (reverse result)
        (let* ([current-sym (car lst)]
               [current-count (hash-ref counts current-sym 0)]
               [new-count (add1 current-count)]
               [new-counts (hash-set counts current-sym new-count)]
               [annotated-pair (list current-sym new-count)])
          (annotate-helper (cdr lst) new-counts (cons annotated-pair result)))))
  
  (annotate-helper lst (hash) '()))

(display (annotate-occurrence '(h e l l o w o r l d)))

((h 1) (e 1) (l 1) (l 2) (o 1) (w 1) (o 2) (r 1) (l 3) (d 1))

#### (c) Convert a trinary string represented as a list of 0s, 1s, and 2s into a (decimal) number:

```
trinary-to-decimal '(1 0 2 1 0)) ; ==> 102
```

In [63]:
(define (trinary-to-decimal lst)
  (define (convert-helper lst acc)
    (if (null? lst)
        acc
        (convert-helper (cdr lst) 
                        (+ (* 3 acc) (car lst)))))
  
  (convert-helper lst 0))

(display (trinary-to-decimal '(1 0 2 1 0)))

102

#### (d) Return the 3rd to last symbol in a list (you may assume it has enough symbols):

```
(third-to-last '(1 1 1 0 1 1)) ; ==> 0
(third-to-last '(s y m b o l)) ; ==> 'b
```

In [67]:
(define (third-to-last lst)
  (define (find-third-to-last lst)
    (if (= (length lst) 3)
        (car lst)
        (find-third-to-last (cdr lst))))
  
  (if (< (length lst) 3)
      (error "List must have at least 3 elements")
      (find-third-to-last lst)))

(display (third-to-last '(1 1 1 0 1 1)))
(newline)
(display (third-to-last '(s y m b o l)))

0
b

#### (e) Decrement a binary number. Decrementing zero should produce zero:

```
(decrement '(1 0 1 1 0)) ; ==> '(1 0 1 0 1)
(decrement '(1 0 0 0 0)) ; ==> '(1 1 1 1)
(decrement '(0)) ; ==> '(0)
```

In [69]:
(define (decrement binary-list)
  (define (decrement-helper lst carry)
    (cond
      [(null? lst) '()]
      [(= carry 1)
       (if (= (car lst) 0)
           (cons 1 (decrement-helper (cdr lst) 1))
           (cons 0 (decrement-helper (cdr lst) 0)))]
      [else (cons (car lst) (decrement-helper (cdr lst) 0))]))

  (define (remove-leading-zeros lst)
    (cond
      [(null? lst) '(0)]
      [(and (= (car lst) 0) (not (null? (cdr lst)))) (remove-leading-zeros (cdr lst))]
      [else lst]))

  (if (equal? binary-list '(0))
      '(0)
      (remove-leading-zeros (reverse (decrement-helper (reverse binary-list) 1)))))

(display (decrement '(1 0 1 1 0)))
(newline)
(display (decrement '(1 0 0 0 0)))
(newline)
(display (decrement '(0)))

(1 0 1 0 1)
(1 1 1 1)
(0)

### 2. Implement in Racket a function `max-and-sum` that computes a maximum and a sum of a list of numbers. For example, ```(max-and-sum (list 6 2 4 1))``` should compute `'(6 13)`.

#### (a) Implement `max-and-sum` using explicit recursion (i.e. **without** higher-order functions).

In [75]:
(define (max-and-sum lst)
  (if (null? lst)
      (list 0 0)  ; Base case: empty list returns (0 0)
      (let* ([rest-result (max-and-sum (cdr lst))]
             [current-max (car rest-result)]
             [current-sum (cadr rest-result)]
             [new-max (max (car lst) current-max)]
             [new-sum (+ (car lst) current-sum)])
        (list new-max new-sum))))

(display (max-and-sum (list 6 2 4 1)))

(6 13)

#### (b) Use the *Substitution Model* to verify that ```(second (max-and-sum (list x y z)))``` is equal to ```(+ x y z)``` for all `x`, `y`, and `z`.

Substitution steps:

```racket
(second (max-and-sum (list x y z)))
(second (let* ([rest-result (max-and-sum (list y z))]
               [current-max (car rest-result)]
               [current-sum (cadr rest-result)]
               [new-max (max x current-max)]
               [new-sum (+ x current-sum)])
          (list new-max new-sum)))
```

Now, let's expand ```(max-and-sum (list y z))```:

```racket
(second (let* ([rest-result (let* ([inner-rest-result (max-and-sum (list z))]
                                   [inner-current-max (car inner-rest-result)]
                                   [inner-current-sum (cadr inner-rest-result)]
                                   [inner-new-max (max y inner-current-max)]
                                   [inner-new-sum (+ y inner-current-sum)])
                              (list inner-new-max inner-new-sum))]
               [current-max (car rest-result)]
               [current-sum (cadr rest-result)]
               [new-max (max x current-max)]
               [new-sum (+ x current-sum)])
          (list new-max new-sum)))
```

Expanding ```(max-and-sum (list z))```:

```racket
(second (let* ([rest-result (let* ([inner-rest-result (let* ([innermost-rest-result (max-and-sum '())]
                                                             [innermost-current-max (car innermost-rest-result)]
                                                             [innermost-current-sum (cadr innermost-rest-result)]
                                                             [innermost-new-max (max z innermost-current-max)]
                                                             [innermost-new-sum (+ z innermost-current-sum)])
                                                        (list innermost-new-max innermost-new-sum))]
                                   [inner-current-max (car inner-rest-result)]
                                   [inner-current-sum (cadr inner-rest-result)]
                                   [inner-new-max (max y inner-current-max)]
                                   [inner-new-sum (+ y inner-current-sum)])
                              (list inner-new-max inner-new-sum))]
               [current-max (car rest-result)]
               [current-sum (cadr rest-result)]
               [new-max (max x current-max)]
               [new-sum (+ x current-sum)])
          (list new-max new-sum)))
```

Now, `(max-and-sum '())` returns `(0 0)`. Substituting and simplifying:

```racket
(second (let* ([rest-result (let* ([inner-rest-result (list (max z 0) (+ z 0))]
                                   [inner-current-max (max z 0)]
                                   [inner-current-sum (+ z 0)]
                                   [inner-new-max (max y (max z 0))]
                                   [inner-new-sum (+ y z)])
                              (list inner-new-max inner-new-sum))]
               [current-max (max y (max z 0))]
               [current-sum (+ y z)]
               [new-max (max x (max y (max z 0)))]
               [new-sum (+ x (+ y z))])
          (list new-max new-sum)))
```

Finally, we obtain:

```racket
(second (list (max x (max y (max z 0))) (+ x (+ y z))))
(+ x (+ y z))
```

Which is equivalent to `(+ x y z)`, as required.

#### (c) Argue whether tail recursion can be used to optimize your implementation.

Tail recursion can be used to optimize the implementation by passing along **accumulators** that keep track of the maximum and sum as the recursion progresses. Instead of computing the maximum and sum after the recursive call returns, we can keep updating these values as we make the recursive call.

To implement this, we can introduce a helper function `max-and-sum-helper` that carries two accumulators: `current-max` and `current-sum`.

- As we recurse through the list, we update these accumulators directly, passing them to the next recursive call.

- When we reach the base case (an empty list), we return the accumulated values as the result.

By making this tail-recursive transformation, the function could handle larger lists without consuming additional stack space.

### 3. Consider the following definitions in Racket:

```racket
(define (dinc n) (+ n 2))

(define (f n)
    (cond
        [(>= n 10) (- n 10)]
        [else (* (f (dinc (dinc n))) (f (dinc n)))]))
```

Using the *Substitution Model*, explain step-by-step how the following expression is
computed (you can evaluate cond-expressions immediately, but evaluation of function calls to `f`
and `dinc` have to be explicit):
    `(f 3)`

#### **Solution**:

#### Step 1.

```
(define (f 3)
    (cond
        [(>= 3 10) (- 3 10)]
        [else (* (f (dinc (dinc 3))) (f (dinc 3)))]))
```

Since 3 is less than 10, we evaluate the `else` branch in the `cond` expression:
`(* (f (dinc (dinc 3))) (f (dinc 3)))`

We now have two recursive calls to `f` to evaluate: `(f (dinc (dinc 3)))` and `(f (dinc 3))`

#### Step 2.

First, evaluate the `dinc` expression:
```
(dinc 3) (+ 3 2)
(dinc 3) (5)
```

And `(dinc (dinc 3))` is the same as `(dinc (+ 3 2))` and equals `(dinc 5)`

Meanwhile `(dinc 5)` is `(+ 5 2)` which is 7

So now we have:

`(* (f 7) (f 5))`

#### Step 3.

Evaluating `(f 7)` and `(f 5)`

##### 1. Evaluating `(f 7)`

   Since 7 is less than 10, we evaluate the `else` branch:

   `(* (f (dinc (dinc 7))) (f (dinc 7)))`

   Now, evaluate the `dinc` expressions, like in Step 2:

   - `(dinc 7)` is `(+ 7 2)` = 9

   - `(dinc (dinc 7))` is `(dinc (+ 7 2))` is `(dinc 9)` and it equals 11

   Thus, we get: `(* (f 11) (f 9))` is our `(f 7)`

##### 2. Evaluating `(f 5)`

   Since 5 is less than 10, we evaluate the `else` branch:

   `(* (f (dinc (dinc 5))) (f (dinc 5)))`

   Now, evaluate the `dinc` expressions:

   We already know from Step 2. that `(dinc 5)` is 7

   And `(dinc (dinc 5))` would be `(dinc 7)` which is 9 (evaluated in Step 3.1)

   Thus, we obtain `(* (f 9) (f 7))` for `(f 5)`

#### Step 4.

Evaluating the recursive calls to `f`

##### 1. Evaluating `(f 11)`

Since 11 > 10, we return 11 - 10 = 1: `(>= 11 10) (- 11 10)` is 1

##### 2. Evaluating `(f 9)`

Since 9 is less than 10, we evaluate the `else` branch:

`(* (f (dinc (dinc 9))) (f (dinc 9)))`

Now, evaluate the `dinc` expressions:

- `(dinc 9)` is 11 (Evaluated in Step 3.1)

- `(dinc (dinc 9))` is `(dinc 11)` is `(dinc (+ 11 2))` is 13

Thus, we get:

`(f 9)` is `(* (f 13) (f 11))`

- Evaluating `(f 13)`

  Since 13 >= 10, we return 13 - 10 = 3: `(>= 13 10) (- 13 10)` is 3

- Evaluating `(f 11)` (as before)

  `(f 11) = 1`

So, `(f 9)` is the same as `(* 3 1) = 3`

Now let's substitute back:

##### 3. Substituting into `(f 7)`

`(f 7) = (* (f 11) (f 9)) = (* 1 3) = 3`

##### 4. Substituting into `(f 5)`

`(f 5) = (* (f 9) (f 7)) = (* 3 3) = 9`

#### Step 5.

Now that we have the results for `(f 7)` and `(f 5)`

Since `(f 3)` is `(* (f 7) (f 5))` (From Step 2.) we substitute:

`(f 3) = (* (f 7) (f 5)) = (* 3 9) = 27`

#### **Final Result**:

`(f 3)` is **27**